In [1]:
import numpy as np
import networkx as nx
import time
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
def generate_graph(n):
    """
    Генерация связного разреженного неориентированного графа с подграфами K7 и K4,5.
    Граф будет иметь:
      - Подграф K7 на первых 7 вершинах.
      - Подграф K4,5 на следующих 9 вершинах (4 вершины в первой доле и 5 во второй).
      - Остовное дерево для обеспечения связности оставшихся вершин.
      - Дополнительные случайные рёбра для достижения нужной разреженности (средняя степень ~√n).
    """
    G = nx.Graph()

    # Добавляем все вершины
    G.add_nodes_from(range(n))

    # Создаем подграф K7 (первые 7 вершин)
    k7_nodes = list(range(7))
    for i in tqdm(k7_nodes, desc="Создание K7", leave=False):
        for j in k7_nodes:
            if i < j:
                G.add_edge(i, j, weight=np.random.uniform(0.1, 1.0))

    # Создаем подграф K4,5 (следующие 9 вершин)
    # Первая доля: 4 вершины, вторая доля: 5 вершин
    k45_nodes_a = list(range(7, 11))  # 4 вершины
    k45_nodes_b = list(range(11, 16))  # 5 вершин
    for i in tqdm(k45_nodes_a, desc="Создание K4,5 (первая доля)", leave=False):
        for j in k45_nodes_b:
            G.add_edge(i, j, weight=np.random.uniform(0.1, 1.0))

    # Обеспечиваем связность: строим остовное дерево для оставшихся вершин (от 16 до n-1)
    if n > 16:
        remaining_nodes = list(range(16, n))
        for i in tqdm(range(1, len(remaining_nodes)), desc="Построение остова", leave=False):
            u = remaining_nodes[i - 1]
            v = remaining_nodes[i]
            G.add_edge(u, v, weight=np.random.uniform(0.1, 1.0))

    # Добавляем случайные ребра для достижения нужной плотности
    # Требуемое число ребер приближенно равно n*sqrt(n)/2 (для неориентированного графа)
    target_edges = int(n * np.sqrt(n)) // 2
    current_edges = G.number_of_edges()

    with tqdm(total=target_edges, desc="Добавление случайных рёбер", leave=False) as pbar:
        pbar.update(current_edges)
        while current_edges < target_edges:
            u = np.random.randint(0, n)
            v = np.random.randint(0, n)
            if u != v and not G.has_edge(u, v):
                G.add_edge(u, v, weight=np.random.uniform(0.1, 1.0))
                current_edges += 1
                pbar.update(1)

    return G

In [3]:
def visualize_subgraphs(G):
    """
    Визуализация подграфов:
      - K7 (первые 7 вершин)
      - K4,5 (вершины с 7 по 15)
    """
    plt.figure(figsize=(15, 6))

    # Визуализация K7
    plt.subplot(121)
    k7 = G.subgraph(range(7))
    nx.draw(k7, with_labels=True, node_color='lightblue', node_size=500)
    plt.title("Подграф K7")

    # Визуализация K4,5
    plt.subplot(122)
    k45 = G.subgraph(range(7, 16))
    # Используем bipartite layout для визуализации двудольного графа K4,5
    pos = nx.bipartite_layout(k45, nodes=range(7, 11))
    nx.draw(k45, pos, with_labels=True, node_color='lightgreen', node_size=500)
    plt.title("Подграф K4,5")

    plt.tight_layout()
    plt.savefig("subgraphs.png")
    plt.close()

In [4]:
def floyd_warshall_vectorized(G):
    """
    Векторизированная реализация алгоритма Флойда–Уоршелла с использованием NumPy.
    Возвращает матрицу расстояний и "эквивалентное" количество итераций (n*n за каждый шаг).
    """
    n = G.number_of_nodes()
    # Инициализация матрицы расстояний
    dist = np.full((n, n), np.inf)
    np.fill_diagonal(dist, 0)

    # Заполнение матрицы для ребер
    for u, v, data in G.edges(data=True):
        dist[u, v] = data['weight']
        dist[v, u] = data['weight']

    iterations = 0
    # Векторизированное обновление: для каждого k обновляем всю матрицу за один шаг
    for k in tqdm(range(n), desc="Флойд–Уоршелл (векторизирован)", leave=False):
        dist = np.minimum(dist, dist[:, k:k + 1] + dist[k:k + 1, :])
        iterations += n * n  # логически эквивалентно тройному циклу
    return dist, iterations

In [5]:
def ford_bellman(G, start):
    """
    Реализация алгоритма Форда–Беллмана для поиска кратчайших путей от стартовой вершины до всех остальных.
    Возвращает массив расстояний и количество выполненных итераций.
    """
    n = G.number_of_nodes()
    dist = np.full(n, np.inf)
    dist[start] = 0

    # Так как граф неориентированный, добавляем рёбра в обоих направлениях
    edges = list(G.edges(data=True)) + [(v, u, d) for u, v, d in G.edges(data=True)]

    iterations = 0
    for i in tqdm(range(n - 1), desc="Форд-Беллман", leave=False):
        relaxed = False
        for u, v, data in edges:
            if dist[u] + data['weight'] < dist[v]:
                dist[v] = dist[u] + data['weight']
                relaxed = True
            iterations += 1
        if not relaxed:
            break  # Если ни одно расстояние не обновилось, завершаем досрочно
    return dist, iterations

In [6]:
def plot_distribution(distances, title):
    """
    Визуализация распределения найденных кратчайших расстояний.
    Строится гистограмма и сохраняется в файл.
    """
    plt.figure(figsize=(10, 6))
    plt.hist(distances, bins=50, color='skyblue', edgecolor='black')
    plt.title(title)
    plt.xlabel("Расстояние")
    plt.ylabel("Количество вершин")
    plt.grid(True, alpha=0.3)
    filename = f"{title.replace(' ', '_')}.png"
    plt.savefig(filename)
    plt.close()

In [7]:
def analyze_graph(n):
    """
    Функция анализа графа заданного размера.
    Генерирует граф, проверяет его свойства, выполняет алгоритмы Флойда–Уоршелла и Форда–Беллмана,
    а также визуализирует распределение расстояний.
    """
    print(f"\nАнализ графа с {n} вершинами")
    print("Генерация графа...")
    G = generate_graph(n)

    # Визуализация подграфов только для небольших графов
    if n <= 100:
        visualize_subgraphs(G)

    # Проверка свойств графа
    print("\nПроверка свойств:")
    print(f"Связность: {nx.is_connected(G)}")
    print(f"Количество вершин: {G.number_of_nodes()}")
    print(f"Количество ребер: {G.number_of_edges()}")
    avg_degree = 2 * G.number_of_edges() / G.number_of_nodes()
    print(f"Средняя степень: {avg_degree:.2f}")
    print(f"Теоретическая средняя степень (√n): {np.sqrt(n):.2f}")

    # Проверка наличия подграфов: K7 и K4,5
    has_k7 = any(len(clique) >= 7 for clique in nx.find_cliques(G))
    print(f"Содержит K7: {has_k7}")
    print("Содержит K4,5: True (встроен явно)")

    # Анализ алгоритмов
    # Запуск алгоритма Флойда-Уоршелла для графов с n <= 8000
    if n <= 8000:
        print("\nЗапуск алгоритма Флойда-Уоршелла...")
        start_time = time.time()
        fw_dist, fw_iter = floyd_warshall_vectorized(G)
        fw_time = time.time() - start_time
        print(f"Время выполнения Флойда-Уоршелла: {fw_time:.2f} сек")
        print(f"Количество итераций: {fw_iter}")
        print(f"Теоретическая сложность O(n^3): {n ** 3}")
    else:
        print("\nАлгоритм Флойда-Уоршелла не выполняется для n > 8000")

    # Запуск алгоритма Форда-Беллмана
    print("\nЗапуск алгоритма Форда-Беллмана...")
    start_time = time.time()
    fb_dist, fb_iter = ford_bellman(G, 0)
    fb_time = time.time() - start_time
    print(f"Время выполнения Форда-Беллмана: {fb_time:.2f} сек")
    print(f"Количество итераций: {fb_iter}")
    print(f"Теоретическая сложность O(nm): {n * G.number_of_edges()}")
    print(f"Максимальное расстояние от вершины 0: {np.max(fb_dist):.2f}")

    # Визуализация распределения расстояний для графов с n <= 20000
    if n <= 20000:
        plot_distribution(fb_dist, f"Распределение расстояний (n={n})")
        print(f"Гистограмма распределения сохранена как 'Распределение_расстояний_(n={n}).png'")

In [8]:
# Размеры графов для анализа
sizes = [1200, 3200, 8000, 20000, 29000]

for size in sizes:
    analyze_graph(size)


Анализ графа с 1200 вершинами
Генерация графа...



Проверка свойств:
Связность: True
Количество вершин: 1200
Количество ребер: 20784
Средняя степень: 34.64
Теоретическая средняя степень (√n): 34.64
Содержит K7: True
Содержит K4,5: True (встроен явно)

Запуск алгоритма Флойда-Уоршелла...


Время выполнения Флойда-Уоршелла: 5.50 сек
Количество итераций: 1728000000
Теоретическая сложность O(n^3): 1728000000

Запуск алгоритма Форда-Беллмана...


Время выполнения Форда-Беллмана: 0.08 сек
Количество итераций: 207840
Теоретическая сложность O(nm): 24940800
Максимальное расстояние от вершины 0: 1.07
Гистограмма распределения сохранена как 'Распределение_расстояний_(n=1200).png'

Анализ графа с 3200 вершинами
Генерация графа...



Проверка свойств:
Связность: True
Количество вершин: 3200
Количество ребер: 90509
Средняя степень: 56.57
Теоретическая средняя степень (√n): 56.57
Содержит K7: True
Содержит K4,5: True (встроен явно)

Запуск алгоритма Флойда-Уоршелла...


...

Время выполнения Форда-Беллмана: 3.26 сек
Количество итераций: 3577700
Теоретическая сложность O(nm): 2862160000
Максимальное расстояние от вершины 0: 0.71
Гистограмма распределения сохранена как 'Распределение_расстояний_(n=8000).png'

Анализ графа с 20000 вершинами
Генерация графа...



Проверка свойств:
Связность: True
Количество вершин: 20000
Количество ребер: 1414213
Средняя степень: 141.42
Теоретическая средняя степень (√n): 141.42
Содержит K7: True
Содержит K4,5: True (встроен явно)

Алгоритм Флойда-Уоршелла не выполняется для n > 8000

Запуск алгоритма Форда-Беллмана...


Время выполнения Форда-Беллмана: 11.99 сек
Количество итераций: 14142130
Теоретическая сложность O(nm): 28284260000
Максимальное расстояние от вершины 0: 0.69
Гистограмма распределения сохранена как 'Распределение_расстояний_(n=20000).png'

Анализ графа с 29000 вершинами
Генерация графа...



Проверка свойств:
Связность: True
Количество вершин: 29000
Количество ребер: 2469261
Средняя степень: 170.29
Теоретическая средняя степень (√n): 170.29
Содержит K7: True
Содержит K4,5: True (встроен явно)

Алгоритм Флойда-Уоршелла не выполняется для n > 8000

Запуск алгоритма Форда-Беллмана...


Время выполнения Форда-Беллмана: 21.52 сек
Количество итераций: 24692610
Теоретическая сложность O(nm): 71608569000
Максимальное расстояние от вершины 0: 0.67
